This time we will correct the latest code version of the Female Daily web scrapping. In the first attempt I tried to scrape all the information with find_all in the first place. It implies that i can scrape information that is filled by "ALL" user. That's  because the program arange the record per element automatically as the order of the record itself. It means that if we have 1 user that doesn't fill 1 information it will disintegrate the order of the record.

This new version of code, it looks up the detail info per user first, so we get the informations per record correct. I also will return "None" or Missing value if the user didn't fill one or more element. 

# Import Modules

In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime



In [25]:
#dir_path = os.getcwd()
startTime = datetime.now()

# TO USE SELENIUM

In [26]:
#impot module
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd

In [58]:
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": r"C:/Users/user/Documents/belajar/",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": False
})

In [59]:
# Get the url (Page 1)
hal = 1
url = 'https://reviews.femaledaily.com/products/lips/lipstick/dear-me-beauty/perfect-matte-lip-coat?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page={hal}'.format(hal=hal)
driver = webdriver.Chrome("C:/Users/istyw/Documents/belajar/Crawling Female Daily with BeautifulSoup/chromedriver.exe",chrome_options=chrome_options)
driver.get(url)

C:\Users\istyw\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [60]:
uname=[]
skintype=[]
oiltype=[]
shade=[]
tone=[]
age=[]
rating=[]
comment=[]
date=[]
rekomen=[]
usage_period=[]
purchase_point=[]

In [67]:
#How to read more comment
more=driver.find_elements_by_css_selector("span[class='read-more']")
for ele in more:
    driver.execute_script("arguments[0].click();", ele)
    # Wrangling HTML with BeautifulSoup

In [66]:
#while True:


#Get Detail  info
soup = BeautifulSoup(driver.page_source,'html.parser')
detailinfo=soup.find_all('div',class_='review-detail d-flex')

#Loop to get info per user review

for elem in detailinfo:
    #Get username
    Username = elem.find('p',class_='username')
    uname.append(Username.text)
    #Get Skintype 
    skin = elem.find('p',class_='skin').text

    #Parsing skin type
    skin2=skin.split(",")
    skintype.append(skin2)
    oiltype.append(skin2[0])
    shade.append(skin2[1])
    tone.append(skin2[2])
    age.append(skin2[3])

    #Get rating

    star = elem.find('span',class_='cardrv-starlist')
    bintang=star.find_all('i',{'class':lambda x: x and "icon-ic_big_star_full" in x.split()})
    rating.append(len(bintang))

    #Get Comment
    #PS I Can not click "Read More" on the comment section. Is it just me? :(
    komen = elem.find('p', class_="text-content")
    comment.append(komen.find('span').text)

    #Get Date review
    daterev=elem.find('p', class_="date review-date")
    date.append(daterev.text)

    #Get Recommended or not
    recinfo=elem.find('p', class_="recommend")
    if recinfo == None:
        rekomen.append("None")
    else :
        if "doesn't recommend" in recinfo.find('b').text:
            rekomen.append("Not Recommend")
        else:
            rekomen.append("Recommend")

    #Get Usage Period and Purchase Point
    infowrapper=elem.find('div', class_="information-wrapper")
    if infowrapper==None:
        usage_period.append("None")
        purchase_point.append("None")
    else :
        info2=infowrapper.find_all('b')
        usage_period.append(info2[0].text)
        purchase_point.append(info2[1].text)






In [65]:
#How to click Next Page 
try :
    next=driver.find_element_by_css_selector("a[id='id_next_page']")
    driver.execute_script("arguments[0].click();", next)
except:
    driver.quit()

    #driver.find_element_by_id("id_next_page").click


#I DID IT OMG

In [49]:
df = pd.DataFrame(list(zip(uname, oiltype,shade, tone, age, rating,comment,date,rekomen,usage_period,purchase_point )), 
               columns =["username", "oiltype","shade", "tone", "age", "rating","comment","date","recommend","usage_period","purchase_point"])

In [57]:
len(comment)

20

In [ ]:
print( datetime.now() - startTime)